In [16]:
import sys
import tensorflow.keras
import pandas as pd
import sklearn as sk
import tensorflow as tf
import keras
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from matplotlib import pyplot
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from math import sqrt
import numpy as np
from sklearn.preprocessing import MinMaxScaler

In [17]:
df_stable = pd.read_excel('Stable_Data.xlsx')
df_stable_US = df_stable.loc[(df_stable['arm'] == 1)]
df_stable_WN = df_stable.loc[(df_stable['arm'] == 2)]

#df = pd.read_excel('Copy of ROS March 2022 data.xlsx', sheet_name='TFI all',skiprows=1)
#df=df.head(100) 

In [18]:
def model_lstm(df):
    ## Separate dependent and independent features 
    X=df.drop(columns=["resp_12","6w_status","12w_status"]).values  #Feature Matrix
    y = df["resp_12"].values
   
    #Train Test Split
    x_train, x_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=0)

    # Reshape input to be 3D for LSTM[samples, timesteps, features]
    x_train = x_train.reshape((x_train.shape[0],1,x_train.shape[1]))
    x_test = x_test.reshape((x_test.shape[0], 1, x_test.shape[1]))
    print(x_train.shape,y_train.shape, x_test.shape, y_test.shape)
    
    # design network
    model = Sequential()
    model.add(LSTM(30, input_shape=(x_train.shape[1], x_train.shape[2])))
    model.add(Dense(1))
   
    # compile and fit
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),loss=tf.keras.losses.MeanSquaredError())
    model.fit(x_train, y_train, epochs=100, batch_size=7, validation_data=(x_test, y_test), verbose=2, shuffle=False)
     
    # make a prediction
    yhat = model.predict(x_test)
    print(y_test)
    print(yhat)
    
    # calculate MAE
    mae = round(mean_absolute_error(y_test, yhat),2)
    return mae

def model_mlp(df):
    ## Separate dependent and independent features
    X=df.drop(columns=["resp_12","6w_status","12w_status"]).values  #Feature Matrix
    y = df["resp_12"].values

    #Train Test Split
    x_train, x_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=0)

    # Prepare input for mlp
    input_size = x_train.shape[0]
    batch_size = 7
    train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
    train_dataset = train_dataset.shuffle(input_size)
    train_dataset = train_dataset.batch(batch_size).cache()
    train_dataset = train_dataset.prefetch(tf.data.experimental.AUTOTUNE)
    
    # design network
    model = Sequential()
    model.add(Dense(20, activation='tanh', input_shape=(x_train.shape[1],)))
    model.add(Dense(1, activation='linear'))
    model.summary()

    # compile and fit
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),loss=tf.keras.losses.MeanSquaredError())
    model.fit(train_dataset, epochs=100)
    
    # make a prediction
    yhat=model.predict(x_test)
    print(y_test)
    print(yhat)

    # calculate MAE
    mae = round(mean_absolute_error(y_test, yhat),2)
    return mae


def data_sample(Model,Sample):
    data= df_stable_US if Sample=='US' else df_stable_WN if Sample=='WN' else df_stable 
    mae= model_lstm(data) if Model=='LSTM' else model_mlp(data) if Model=='MLP' else "Work in progress"  
    output = "Mean Absolute Error of TFI regression is: %s" % (mae)
    return output

In [19]:
import gradio as gr   # pip install gradio

iface = gr.Interface(
    data_sample,
    inputs=[gr.inputs.Radio(["LSTM","MLP"],label="Choose a Model"),gr.inputs.Radio(["US+WN","US","WN"],label="Select the Sample")],
    #outputs="number",
    outputs=["text"],
    title="AI in Tinnitus",
    description="Please select the Model & Sample, then click Submit to see the Results!"
)

iface.launch(share=False)

Running on local URL:  http://127.0.0.1:7868/

To create a public link, set `share=True` in `launch()`.


(<fastapi.applications.FastAPI at 0x179791e20>, 'http://127.0.0.1:7868/', None)

(42, 1, 8) (42,) (19, 1, 8) (19,)
Epoch 1/100


2022-04-16 19:40:37.249062: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-04-16 19:40:37.331977: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-04-16 19:40:37.358870: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


6/6 - 1s - loss: 517.5272 - val_loss: 801.9384 - 818ms/epoch - 136ms/step
Epoch 2/100
6/6 - 0s - loss: 492.4762 - val_loss: 772.7029 - 51ms/epoch - 8ms/step
Epoch 3/100
6/6 - 0s - loss: 471.5807 - val_loss: 748.5040 - 48ms/epoch - 8ms/step
Epoch 4/100


...

2022-04-16 19:41:08.997298: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_7 (Dense)             (None, 20)                180       
                                                                 
 dense_8 (Dense)             (None, 1)                 21        
                                                                 
Total params: 201
Trainable params: 201
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
6/6 [==============================] - 0s 4ms/step - loss: 519.2745
Epoch 2/100
6/6 [==============================] - 0s 4ms/step - loss: 471.6452
Epoch 3/100
6/6 [==============================] - 0s 4ms/step - loss: 439.8938
Epoch 4/100
6/6 [==============================] - 0s 4ms/step - loss: 417.4728
Epoch 5/100
6/6 [==============================] - 0s 4ms/step - loss: 394.4713
Epoch 6/100
1/6 [====>.........................] - E

2022-04-16 19:41:12.071362: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


6/6 [==============================] - 0s 4ms/step - loss: 374.6702
Epoch 7/100
6/6 [==============================] - 0s 4ms/step - loss: 355.2670
Epoch 8/100
6/6 [==============================] - 0s 4ms/step - loss: 339.4664
Epoch 9/100
6/6 [==============================] - 0s 4ms/step - loss: 322.8850
Epoch 10/100
6/6 [==============================] - 0s 4ms/step - loss: 309.4210
Epoch 11/100
6/6 [==============================] - 0s 3ms/step - loss: 295.3583
Epoch 12/100
6/6 [==============================] - 0s 4ms/step - loss: 282.7233
Epoch 13/100
6/6 [==============================] - 0s 4ms/step - loss: 271.4510
Epoch 14/100
6/6 [==============================] - 0s 3ms/step - loss: 259.8565
Epoch 15/100
6/6 [==============================] - 0s 3ms/step - loss: 252.6281
Epoch 16/100
6/6 [==============================] - 0s 3ms/step - loss: 242.9047
Epoch 17/100
6/6 [==============================] - 0s 4ms/step - loss: 231.7540
Epoch 18/100
6/6 [==========================

2022-04-16 19:41:14.656553: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
